In [1]:
import os
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")

## RAG using multiple data sources

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki_tool.name

'wikipedia'

In [5]:
# Creating another tool (custom tool)
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

In [6]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

db = FAISS.from_documents(documents, OllamaEmbeddings(model="tinyllama"))
retriever = db.as_retriever()


In [7]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025629B46CB0>)

In [8]:
# Creating a retriver tool for retrieving documents
from langchain.tools.retriever import create_retriever_tool

In [9]:
retriever_tool = create_retriever_tool(retriever, "langsmith_search",
                       "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")


In [10]:
retriever_tool.name

'langsmith_search'

In [11]:
# Tool for retrieving documents from arxiv
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [12]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [13]:
arxiv_tool.name

'arxiv'

In [14]:
# Combining all tools
tools = [wiki_tool, retriever_tool, arxiv_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\projects\\langchain_projects\\base\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002561AF8AE60>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025629B46CB0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000002561AF8B010>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langcha

In [23]:

# Agents; for qeuring from these tools
"""
The core idea of agents is to use a language model to choose a sequence of actions to take.
In chains, a sequence of actions is hardcoded (in code). 
In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.
"""
from langchain.agents import create_tool_calling_agent
from langchain_openai import ChatOpenAI
from langchain import hub # Pre-written prompts

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [21]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [24]:
# llm = ChatOllama(model="llama2", temperature=0, keep_alive=-1)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [25]:
# Agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [26]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06

In [27]:
# agent_executor.invoke({"input":"Tell me about Langsmith"})